<a href="https://www.kaggle.com/code/ayoubcherguelaine/tf-with-ir-datasets?scriptVersionId=110728490" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [4]:
pip install --upgrade ir_datasets

Note: you may need to restart the kernel to use updated packages.


In [5]:
#! python -m spacy download fr_core_news_sm
! python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 57.4 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


**process class**

In [6]:
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import spacy
import re
import numpy as np
import math

In [11]:
def RM_Punc(text):
    punctuation_pattern = re.compile('[^A-z ]')
    no_punct_text = [re.sub(punctuation_pattern, '', token) for token in text]
    return ''.join([token for token in no_punct_text if token != ''])

def preSomthing(text):
    text = text.lower()
    text = RM_Punc(text)
    return text
def RM_Stop_Word(tokens):
    # check in lowercase 
    t = [token for token in tokens if token.lower() not in stopwords.words("english")]
      
    return t

nlp = spacy.load('en_core_web_sm')
def Lemm(text):
    
    doc = nlp(text)
    return [i.lemma_ for i in doc]

def PreProcessing(text):
    text = preSomthing(text)
    tokens= Lemm(text)
    return RM_Stop_Word(tokens)

def removeAll(li=[],item=object):
    while( li.count(item) >0 ):
        li.remove(item)

vector=[]
def indexVect(token):
    tokens= token.copy()
    
    index=[]
    som = len(tokens)
    i=0
    
    while (len(tokens)>0):
        i = tokens[0]
        indexVal = tokens.count(i)
        if i not in vector:
            vector.append(i)
        removeAll(tokens,i)    
        index.append([vector.index(i),indexVal/som])
    return index   

**indexation**

In [12]:
text="""# Modélisation du langage Naturel

### Exemple 1 "Programmation procédurale"

En prenant en considération [cette ressource](https://dev.to/amananandrai/language-model-implementation-bigram-model-22ij), il vous est demandé de:

1. Afficher les bigrams
2. Calculer les probabilités 

pour chacune des phrases suivantes:

    - This is my dog
    - I love my dog
    - This is a name
    - I love my name

3. Optimisez le code pour prendre en considération le modèle tri-grams.
4. Recalculez les probabilités. 
5. Analysez les résultats."""
indexVect(PreProcessing(text))

[[0, 0.05172413793103448],
 [1, 0.017241379310344827],
 [2, 0.017241379310344827],
 [3, 0.017241379310344827],
 [4, 0.017241379310344827],
 [5, 0.017241379310344827],
 [6, 0.017241379310344827],
 [7, 0.017241379310344827],
 [8, 0.017241379310344827],
 [9, 0.034482758620689655],
 [10, 0.034482758620689655],
 [11, 0.017241379310344827],
 [12, 0.017241379310344827],
 [13, 0.017241379310344827],
 [14, 0.017241379310344827],
 [15, 0.017241379310344827],
 [16, 0.017241379310344827],
 [17, 0.017241379310344827],
 [18, 0.017241379310344827],
 [19, 0.017241379310344827],
 [20, 0.06896551724137931],
 [21, 0.017241379310344827],
 [22, 0.017241379310344827],
 [23, 0.017241379310344827],
 [24, 0.034482758620689655],
 [25, 0.017241379310344827],
 [26, 0.017241379310344827],
 [27, 0.017241379310344827],
 [28, 0.017241379310344827],
 [29, 0.06896551724137931],
 [30, 0.034482758620689655],
 [31, 0.034482758620689655],
 [32, 0.034482758620689655],
 [33, 0.017241379310344827],
 [34, 0.034482758620689655]

In [13]:
vector

[' ',
 'modlisation',
 'du',
 'langage',
 'naturel',
 'exemple',
 'programmation',
 'procduraleen',
 'prenant',
 'en',
 'considration',
 '[',
 'cette',
 'ressource]httpsdevtoamananandrailanguagemodelimplementationbigrammodelij',
 'il',
 'vous',
 'est',
 'demand',
 'de',
 'afficher',
 'les',
 'bigrams',
 'calculer',
 'probabilits',
 'pour',
 'chacune',
 'des',
 'phrase',
 'suivante',
 '    ',
 'dog',
 'love',
 'name',
 'optimisez',
 'le',
 'code',
 'prendre',
 'modle',
 'trigrams',
 'recalculez',
 'probabilit',
 'analysez',
 'rsultat']

**get score docs**

In [14]:
import ir_datasets
datasetDoc = ir_datasets.load("wikir/en1k")
for doc in datasetDoc.docs_iter():
    doc 

[INFO] If you have a local copy of https://zenodo.org/record/3565761/files/wikIR1k.zip, you can symlink it here to avoid downloading it again: /root/.ir_datasets/downloads/554299bca984640cb283d6ba55753608
[INFO] [starting] https://zenodo.org/record/3565761/files/wikIR1k.zip
[INFO] [finished] https://zenodo.org/record/3565761/files/wikIR1k.zip: [00:01] [165MB] [107MB/s]


In [15]:
datasetQuery = ir_datasets.load("wikir/en1k/test")
for query in datasetQuery.queries_iter():
    query # namedtuple<query_id, text>

In [18]:
docs = []
docsID = []
processing = 1
for i in datasetDoc.docs_iter()[:100000]:
    print(processing,"/20000")  
    docs.append(indexVect(PreProcessing(i.text)))
    docsID.append(i.doc_id)
    processing=processing+1

0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0 /20000
0

In [19]:
len(docsID)

20000

In [20]:
Query=[]
QueryID=[]

for i in datasetQuery.queries_iter():
        Query.append(indexVect(PreProcessing(i.text)))
        QueryID.append(i.query_id)

In [22]:
len(Query)


100

**using cosinus**

In [23]:

def tfCos(dnp, qnp):   
    inner=0.0
    p1=0.0
    p2=0.0
    if len(qnp) == 0 or len(dnp)==0:
        return 0
    for i in qnp[:,:1]:
        
        if i in dnp[:,:1]:
            
            qi= np.where(qnp[:,:1]==i)
            
            di= np.where(dnp[:,:1]==i)
          
            inner = inner + (dnp[di[0]][:,1:2][0][0] * qnp[qi[0]][:,1:2][0][0]  )
            p1= p1 + dnp[di[0]][:,1:2][0][0]**2
            p2= p2 + qnp[qi[0]][:,1:2][0][0]**2
            
    if p1==0.0 or p2 ==0.0:
        return 0  
    cos = (inner) /( math.sqrt(p1) * math.sqrt(p2) )
    return cos    


test cos

In [24]:
Rcos ={}

for i in range(len(Query)):
    R=[]
    for j in range(len(docs)):
    
        iq = np.array( Query[i])
    
        DAi = np.array(docs[j] )
    
        R.append( [ docsID[j] , tfCos( DAi,iq)])
    Rcos[QueryID[i]] =R



In [25]:
len(Rcos['158491'])

20000

In [26]:
scores = []
for score in datasetQuery.scoreddocs_iter():
    
    scores.append(score)
        
len(scores)

10000

In [27]:
sc=[]
for i in scores:
    if i.doc_id in docsID:
        sc.append(i)

In [28]:
treash =Rcos[list(Rcos.keys())[0]]
n = np.array(treash)
len(n)

20000

In [29]:
nbDocsPer=0
nbDocReturne=0
nbDocRetPer=0
for i in list(Rcos.keys()):
    trash = Rcos[i]
    n = np.array(trash)
    for j in sc:
        ind = np.where(n[:,:1] == j.doc_id)
        row = n[ind[0]][0][1]
        score= j.score
        if int(row[0])>0 and score >0:
            nbDocRetPer=nbDocRetPer+1
            
        if int(row[0])>0:
            nbDocReturne=nbDocReturne+1
            
        if score>0:
            nbDocsPer=nbDocsPer+1
       
        
        
       
print(nbDocsPer,nbDocReturne,nbDocRetPer)

42600 2136 2136


**dice**